In [2]:
# MedMeshCNN - TensorFlow Implementation
# Jupyter Notebook으로 각 셀별로 실행 가능하도록 구성

# =============================================================================
# 필요한 라이브러리 import
# =============================================================================

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import json
from typing import List, Tuple, Dict, Optional, Union
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import pickle
import trimesh

print(f"TensorFlow version: {tf.__version__}")

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # 필요한 만큼만 메모리 할당 (동적 할당)
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ {len(gpus)}개의 GPU에 memory growth 설정 완료")
    except RuntimeError as e:
        print(e)


TensorFlow version: 2.9.1
✅ 1개의 GPU에 memory growth 설정 완료


In [ ]:
#import numpy as np
#import os
#import pickle
#import json
#import tensorflow as tf
#from typing import Dict, List, Tuple, Optional, Union
#from pathlib import Pat

class MeshDatasetTF:
    """TensorFlow용 메쉬 데이터셋 클래스"""
    
    def __init__(self, data_dir: str, cache_dir: Optional[str] = None, 
                 max_vertices: int = 5000, max_faces: int = 10000):
        """
        Args:
            data_dir: 메쉬 파일들이 있는 디렉토리
            cache_dir: 전처리된 데이터 캐시 디렉토리
            max_vertices: 최대 정점 수 (패딩/샘플링용)
            max_faces: 최대 면 수 (패딩/샘플링용)
        """
        self.data_dir = Path(data_dir)
        self.cache_dir = Path(cache_dir) if cache_dir else self.data_dir / "cache"
        self.max_vertices = max_vertices
        self.max_faces = max_faces
        
        # 지원하는 파일 형식
        self.supported_formats = ['.obj', '.ply', '.stl', '.off']
        
        # 메쉬 파일 목록 수집
        self.mesh_files = self._collect_mesh_files()
        
        # 캐시 디렉토리 생성
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        
        # 메타데이터 파일
        self.metadata_file = self.cache_dir / "dataset_metadata.json"
        
        print(f"📁 TensorFlow 데이터셋 초기화 완료: {len(self.mesh_files)}개 파일")
    
    def _collect_mesh_files(self) -> List[Path]:
        """메쉬 파일 목록 수집"""
        mesh_files = []
        
        for ext in self.supported_formats:
            mesh_files.extend(self.data_dir.glob(f"**/*{ext}"))
        
        return sorted(mesh_files)
    
    def load_mesh_data(self, file_path: str) -> Dict:
        """개별 메쉬 파일 로딩 (TensorFlow용으로 최적화)"""
        try:
            # Trimesh로 빠르게 로딩
            mesh = trimesh.load(file_path, force='mesh')
            
            # Scene인 경우 첫 번째 메쉬 선택
            if hasattr(mesh, 'geometry'):
                mesh = list(mesh.geometry.values())[0]
            
            vertices = np.array(mesh.vertices, dtype=np.float32)
            faces = np.array(mesh.faces, dtype=np.int32)
            
            # 기본 validation
            if len(vertices) == 0 or len(faces) == 0:
                raise ValueError("빈 메쉬")
            
            mesh_data = {
                'vertices': vertices,
                'faces': faces,
                'file_path': file_path
            }
            
            return mesh_data
            
        except Exception as e:
            print(f"⚠️ 로딩 실패 {file_path}: {e}")
            # 더미 데이터 반환 (학습 중단 방지)
            return self._create_dummy_mesh()
    
    def _create_dummy_mesh(self) -> Dict:
        """로딩 실패 시 더미 메쉬 생성"""
        # 간단한 정사면체
        vertices = np.array([
            [0.0, 0.0, 0.0],
            [1.0, 0.0, 0.0],
            [0.5, 0.866, 0.0],
            [0.5, 0.289, 0.816]
        ], dtype=np.float32)
        
        faces = np.array([
            [0, 1, 2],
            [0, 2, 3],
            [0, 3, 1],
            [1, 3, 2]
        ], dtype=np.int32)
        
        return {
            'vertices': vertices,
            'faces': faces,
            'file_path': 'dummy'
        }
    
    def _preprocess_mesh(self, mesh_data: Dict) -> Dict:
        """메쉬 데이터 전처리"""
        vertices = mesh_data['vertices']
        faces = mesh_data['faces']
        
        # 1. 정점 정규화 (단위 구 안에 맞추기)
        vertices = self._normalize_vertices(vertices)
        
        # 2. 메쉬 크기 조정 (샘플링 또는 패딩)
        vertices, faces = self._resize_mesh(vertices, faces)
        
        # 3. 면 법선 계산
        face_normals = self._compute_face_normals(vertices, faces)
        
        # 4. 엣지 특성 계산
        edge_features = self._compute_edge_features(vertices, faces)
        
        # 5. TensorFlow 텐서로 변환
        processed_data = {
            'vertices': tf.constant(vertices, dtype=tf.float32),      # [max_vertices, 3]
            'faces': tf.constant(faces, dtype=tf.int32),             # [max_faces, 3]
            'normals': tf.constant(face_normals, dtype=tf.float32),   # [max_faces, 3]
            'features': tf.constant(edge_features, dtype=tf.float32), # [max_faces, 5]
            'vertex_mask': tf.constant(len(vertices) > 0, dtype=tf.bool),  # 유효한 정점 마스크
            'face_mask': tf.constant(len(faces) > 0, dtype=tf.bool),       # 유효한 면 마스크
        }
        
        return processed_data
    
    def _normalize_vertices(self, vertices: np.ndarray) -> np.ndarray:
        """정점을 단위 구 안으로 정규화"""
        # 중심점을 원점으로
        center = vertices.mean(axis=0)
        vertices = vertices - center
        
        # 단위 구 안으로 scaling
        max_dist = np.max(np.linalg.norm(vertices, axis=1))
        if max_dist > 0:
            vertices = vertices / max_dist
        
        return vertices
    
    def _resize_mesh(self, vertices: np.ndarray, faces: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """메쉬 크기를 고정 크기로 조정"""
        n_vertices = len(vertices)
        n_faces = len(faces)
        
        # 정점 처리
        if n_vertices > self.max_vertices:
            # 다운샘플링 (균등하게 선택)
            indices = np.linspace(0, n_vertices-1, self.max_vertices, dtype=int)
            vertices = vertices[indices]
            # 면 인덱스 재매핑 필요
            faces = self._remap_faces(faces, indices)
        elif n_vertices < self.max_vertices:
            # 패딩 (마지막 정점 반복 또는 0으로 패딩)
            if n_vertices > 0:
                padding = np.tile(vertices[-1:], (self.max_vertices - n_vertices, 1))
            else:
                padding = np.zeros((self.max_vertices - n_vertices, 3), dtype=np.float32)
            vertices = np.vstack([vertices, padding])
        
        # 면 처리
        n_faces = len(faces)  # 재매핑 후 면 개수
        if n_faces > self.max_faces:
            # 다운샘플링
            indices = np.linspace(0, n_faces-1, self.max_faces, dtype=int)
            faces = faces[indices]
        elif n_faces < self.max_faces:
            # 패딩 (마지막 면 반복 또는 0으로 패딩)
            if n_faces > 0:
                padding = np.tile(faces[-1:], (self.max_faces - n_faces, 1))
            else:
                padding = np.zeros((self.max_faces, 3), dtype=np.int32)
            faces = np.vstack([faces, padding])
        
        return vertices, faces
    
    def _remap_faces(self, faces: np.ndarray, vertex_indices: np.ndarray) -> np.ndarray:
        """정점 다운샘플링 후 면 인덱스 재매핑"""
        # 인덱스 매핑 테이블 생성
        old_to_new = {old_idx: new_idx for new_idx, old_idx in enumerate(vertex_indices)}
        
        # 유효한 면만 선택 (모든 정점이 매핑되는 면)
        valid_faces = []
        for face in faces:
            if all(v in old_to_new for v in face):
                new_face = [old_to_new[v] for v in face]
                valid_faces.append(new_face)
        
        return np.array(valid_faces, dtype=np.int32) if valid_faces else np.array([]).reshape(0, 3)
    
    def _compute_face_normals(self, vertices: np.ndarray, faces: np.ndarray) -> np.ndarray:
        """면 법선 계산"""
        if len(faces) == 0:
            return np.zeros((self.max_faces, 3), dtype=np.float32)
        
        normals = []
        for face in faces:
            if len(np.unique(face)) == 3:  # 유효한 삼각형
                v0, v1, v2 = vertices[face]
                edge1 = v1 - v0
                edge2 = v2 - v0
                normal = np.cross(edge1, edge2)
                norm = np.linalg.norm(normal)
                if norm > 1e-8:
                    normal = normal / norm
                else:
                    normal = np.array([0.0, 0.0, 1.0])
            else:
                normal = np.array([0.0, 0.0, 1.0])
            
            normals.append(normal)
        
        # 패딩
        while len(normals) < self.max_faces:
            normals.append(np.array([0.0, 0.0, 1.0]))
        
        return np.array(normals[:self.max_faces], dtype=np.float32)
    
    def _compute_edge_features(self, vertices: np.ndarray, faces: np.ndarray) -> np.ndarray:
        """엣지 특성 계산 (학습용 최적화)"""
        if len(faces) == 0:
            return np.zeros((self.max_faces, 5), dtype=np.float32)
        
        features = []
        for face in faces:
            if len(np.unique(face)) == 3:
                v0, v1, v2 = vertices[face]
                
                # 간단한 특성들만 계산 (속도 최적화)
                edge1 = v1 - v0
                edge2 = v2 - v0
                edge3 = v2 - v1
                
                # 1. 면적
                area = 0.5 * np.linalg.norm(np.cross(edge1, edge2))
                
                # 2. 둘레
                perimeter = (np.linalg.norm(edge1) + 
                           np.linalg.norm(edge2) + 
                           np.linalg.norm(edge3))
                
                # 3. 종횡비
                edge_lengths = [np.linalg.norm(edge1), np.linalg.norm(edge2), np.linalg.norm(edge3)]
                aspect_ratio = max(edge_lengths) / (min(edge_lengths) + 1e-8)
                
                # 4. 최소 각도
                cos_angle = np.dot(edge1, edge2) / (np.linalg.norm(edge1) * np.linalg.norm(edge2) + 1e-8)
                min_angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
                
                # 5. 중심점 거리
                centroid = (v0 + v1 + v2) / 3.0
                distance = np.linalg.norm(centroid)
                
                features.append([area, perimeter, aspect_ratio, min_angle, distance])
            else:
                features.append([0.0, 0.0, 1.0, 0.0, 0.0])
        
        # 패딩
        while len(features) < self.max_faces:
            features.append([0.0, 0.0, 1.0, 0.0, 0.0])
        
        return np.array(features[:self.max_faces], dtype=np.float32)
    
    def _load_and_process_mesh(self, file_path_tensor):
        """tf.py_function으로 호출될 메쉬 로딩 함수"""
        file_path = file_path_tensor.numpy().decode('utf-8')
        
        # 캐시 확인
        cache_file = self.cache_dir / f"{Path(file_path).stem}_{hash(file_path) % 10000}.pkl"
        
        if cache_file.exists():
            # 캐시에서 로딩
            with open(cache_file, 'rb') as f:
                mesh_data = pickle.load(f)
        else:
            # 원본 파일에서 로딩 및 전처리
            raw_mesh = self.load_mesh_data(file_path)
            mesh_data = self._preprocess_mesh(raw_mesh)
            
            # 캐시에 저장 (numpy 배열로 변환해서 저장)
            cache_data = {k: v.numpy() if hasattr(v, 'numpy') else v 
                         for k, v in mesh_data.items() if k not in ['vertex_mask', 'face_mask']}
            with open(cache_file, 'wb') as f:
                pickle.dump(cache_data, f)
        
        # TensorFlow 텐서로 변환
        return (
            tf.constant(mesh_data['vertices'] if isinstance(mesh_data['vertices'], np.ndarray) else mesh_data['vertices'].numpy(), dtype=tf.float32),
            tf.constant(mesh_data['faces'] if isinstance(mesh_data['faces'], np.ndarray) else mesh_data['faces'].numpy(), dtype=tf.int32),
            tf.constant(mesh_data['normals'] if isinstance(mesh_data['normals'], np.ndarray) else mesh_data['normals'].numpy(), dtype=tf.float32),
            tf.constant(mesh_data['features'] if isinstance(mesh_data['features'], np.ndarray) else mesh_data['features'].numpy(), dtype=tf.float32)
        )
    
    def create_dataset(self, batch_size: int = 8, shuffle: bool = True, 
                      prefetch_buffer: int = tf.data.AUTOTUNE, 
                      num_parallel_calls: int = tf.data.AUTOTUNE) -> tf.data.Dataset:
        """TensorFlow Dataset 생성"""
        
        # 파일 경로 리스트를 Dataset으로 변환
        file_paths = [str(f) for f in self.mesh_files]
        dataset = tf.data.Dataset.from_tensor_slices(file_paths)
        
        if shuffle:
            dataset = dataset.shuffle(buffer_size=len(file_paths))
        
        # 메쉬 로딩 및 전처리
        dataset = dataset.map(
            lambda file_path: tf.py_function(
                func=self._load_and_process_mesh,
                inp=[file_path],
                Tout=[tf.float32, tf.int32, tf.float32, tf.float32]
            ),
            num_parallel_calls=num_parallel_calls
        )
        
        # 출력 형태 지정
        dataset = dataset.map(lambda v, f, n, feat: {
            'vertices': tf.reshape(v, [self.max_vertices, 3]),
            'faces': tf.reshape(f, [self.max_faces, 3]),
            'normals': tf.reshape(n, [self.max_faces, 3]),
            'features': tf.reshape(feat, [self.max_faces, 5])
        })
        
        # 배치 처리
        dataset = dataset.batch(batch_size, drop_remainder=True)
        
        # 성능 최적화
        if prefetch_buffer:
            dataset = dataset.prefetch(prefetch_buffer)
        
        return dataset
    
    def clear_cache(self):
        """캐시 파일들 삭제"""
        import shutil
        if self.cache_dir.exists():
            shutil.rmtree(self.cache_dir)
            self.cache_dir.mkdir(parents=True, exist_ok=True)
        print("🗑️ 캐시 삭제 완료")

def create_mesh_dataset(data_dir: str, batch_size: int = 8, shuffle: bool = True,
                       max_vertices: int = 5000, max_faces: int = 10000, 
                       cache_dir: Optional[str] = None) -> tf.data.Dataset:
    """TensorFlow 메쉬 데이터셋 생성 함수"""
    
    dataset_loader = MeshDatasetTF(
        data_dir=data_dir,
        cache_dir=cache_dir,
        max_vertices=max_vertices,
        max_faces=max_faces
    )
    
    return dataset_loader.create_dataset(batch_size=batch_size, shuffle=shuffle)

# 사용 예시
if __name__ == "__main__":
    # 데이터셋 생성
    train_dataset = create_mesh_dataset(
        data_dir="./mesh_data/train",
        batch_size=16,
        max_vertices=2000,
        max_faces=4000,
        shuffle=True
    )
    
    # 학습 루프 예시
    print("🚀 TensorFlow 데이터셋 테스트")
    for batch_idx, batch in enumerate(train_dataset.take(3)):
        print(f"\nBatch {batch_idx}:")
        print(f"  Vertices shape: {batch['vertices'].shape}")    # (16, 2000, 3)
        print(f"  Faces shape: {batch['faces'].shape}")        # (16, 4000, 3)
        print(f"  Normals shape: {batch['normals'].shape}")    # (16, 4000, 3)
        print(f"  Features shape: {batch['features'].shape}")  # (16, 4000, 5)
    
    # 모델 학습 예시
    """
    model = YourMeshModel()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    @tf.function
    def train_step(batch):
        with tf.GradientTape() as tape:
            predictions = model(batch['vertices'], batch['faces'], batch['features'])
            loss = compute_loss(predictions, targets)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
    
    # 학습
    for epoch in range(num_epochs):
        for batch in train_dataset:
            loss = train_step(batch)
            print(f"Loss: {loss}")
    """

In [ ]:

# =============================================================================
# 데이터 로딩 및 전처리 유틸리티
# =============================================================================

class MeshDataLoader:
    """메쉬 데이터 로딩 및 전처리를 위한 클래스"""

    def __init__(self):
        self.supported_formats = ['.obj', '.ply', '.stl', '.off', '.3mf']
    
    def load_mesh_data(self, file_path: str) -> Dict:
        """개별 메쉬 파일 로딩"""
        file_path = Path(file_path)
        
        if not file_path.exists():
            raise FileNotFoundError(f"메쉬 파일을 찾을 수 없습니다: {file_path}")
        
        if file_path.suffix.lower() not in self.supported_formats:
            raise ValueError(f"지원하지 않는 파일 형식입니다. 지원 형식: {self.supported_formats}")
        
        try:
            # Method 1: trimesh 사용 (권장)
            mesh_data = self._load_with_trimesh(file_path)
            print(f"✅ Trimesh로 로딩 성공: {file_path.name}")
            return mesh_data
            
        except Exception as e:
            print(f"⚠️ Trimesh 로딩 실패: {e}")
            try:
                # Method 2: Open3D 사용 (백업)
                mesh_data = self._load_with_open3d(file_path)
                print(f"✅ Open3D로 로딩 성공: {file_path.name}")
                return mesh_data
                
            except Exception as e2:
                print(f"❌ Open3D 로딩도 실패: {e2}")
                # Method 3: 수동 파싱 (OBJ 파일만)
                if file_path.suffix.lower() == '.obj':
                    mesh_data = self._load_obj_manual(file_path)
                    print(f"✅ 수동 파싱으로 로딩 성공: {file_path.name}")
                    return mesh_data
                else:
                    raise RuntimeError(f"모든 로딩 방법 실패: {file_path}")
    
    def _load_with_trimesh(self, file_path: Path) -> Dict:
        """Trimesh 라이브러리를 사용한 로딩"""
        mesh = trimesh.load(str(file_path))
        
        # 여러 메쉬가 있는 경우 첫 번째 메쉬 사용
        if isinstance(mesh, trimesh.Scene):
            mesh = list(mesh.geometry.values())[0]
        
        vertices = np.array(mesh.vertices, dtype=np.float32)
        faces = np.array(mesh.faces, dtype=np.int32)
        
        # 면 법선 계산
        face_normals = np.array(mesh.face_normals, dtype=np.float32)
        
        # 엣지 특성 계산
        edge_features = self._compute_edge_features(vertices, faces)
        
        mesh_data = {
            'vertices': vertices,  # N x 3
            'faces': faces,        # M x 3
            'features': edge_features,  # M x 5 (면적, 각도 등)
            'normals': face_normals,    # M x 3
            'metadata': {
                'vertex_count': len(vertices),
                'face_count': len(faces),
                'is_watertight': mesh.is_watertight,
                'bounds': mesh.bounds
            }
        }
        
        return mesh_data
    
    def _load_with_open3d(self, file_path: Path) -> Dict:
        """Open3D 라이브러리를 사용한 로딩"""
        mesh = o3d.io.read_triangle_mesh(str(file_path))
        
        if len(mesh.vertices) == 0:
            raise ValueError("빈 메쉬 파일입니다.")
        
        vertices = np.asarray(mesh.vertices, dtype=np.float32)
        faces = np.asarray(mesh.triangles, dtype=np.int32)
        
        # 면 법선 계산
        mesh.compute_triangle_normals()
        face_normals = np.asarray(mesh.triangle_normals, dtype=np.float32)
        
        # 엣지 특성 계산
        edge_features = self._compute_edge_features(vertices, faces)
        
        mesh_data = {
            'vertices': vertices,
            'faces': faces,
            'features': edge_features,
            'normals': face_normals,
            'metadata': {
                'vertex_count': len(vertices),
                'face_count': len(faces),
                'bounds': [vertices.min(axis=0), vertices.max(axis=0)]
            }
        }
        
        return mesh_data
    
    def _load_obj_manual(self, file_path: Path) -> Dict:
        """OBJ 파일 수동 파싱"""
        vertices = []
        faces = []
        
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('v '):  # 정점
                    coords = list(map(float, line.split()[1:4]))
                    vertices.append(coords)
                elif line.startswith('f '):  # 면
                    # f v1/vt1/vn1 v2/vt2/vn2 v3/vt3/vn3 형식 처리
                    face_data = line.split()[1:]
                    face_indices = []
                    for vertex_data in face_data:
                        # v/vt/vn 또는 v//vn 또는 v 형식
                        vertex_index = int(vertex_data.split('/')[0]) - 1  # OBJ는 1-indexed
                        face_indices.append(vertex_index)
                    
                    # 삼각형이 아닌 경우 삼각분할
                    if len(face_indices) == 3:
                        faces.append(face_indices)
                    elif len(face_indices) > 3:
                        # 팬 삼각분할
                        for i in range(1, len(face_indices) - 1):
                            faces.append([face_indices[0], face_indices[i], face_indices[i + 1]])
        
        vertices = np.array(vertices, dtype=np.float32)
        faces = np.array(faces, dtype=np.int32)
        
        # 면 법선 계산
        face_normals = self._compute_face_normals(vertices, faces)
        
        # 엣지 특성 계산
        edge_features = self._compute_edge_features(vertices, faces)
        
        mesh_data = {
            'vertices': vertices,
            'faces': faces,
            'features': edge_features,
            'normals': face_normals,
            'metadata': {
                'vertex_count': len(vertices),
                'face_count': len(faces),
                'bounds': [vertices.min(axis=0), vertices.max(axis=0)]
            }
        }
        
        return mesh_data
    
    def _compute_face_normals(self, vertices: np.ndarray, faces: np.ndarray) -> np.ndarray:
        """면 법선 벡터 계산"""
        face_normals = []
        
        for face in faces:
            v0, v1, v2 = vertices[face]
            edge1 = v1 - v0
            edge2 = v2 - v0
            normal = np.cross(edge1, edge2)
            
            # 정규화
            norm = np.linalg.norm(normal)
            if norm > 0:
                normal = normal / norm
            else:
                normal = np.array([0.0, 0.0, 1.0])  # 기본값
            
            face_normals.append(normal)
        
        return np.array(face_normals, dtype=np.float32)
    
    def _compute_edge_features(self, vertices: np.ndarray, faces: np.ndarray) -> np.ndarray:
        """엣지/면 특성 계산"""
        features = []
        
        for face in faces:
            v0, v1, v2 = vertices[face]
            
            # 1. 면적
            edge1 = v1 - v0
            edge2 = v2 - v0
            area = 0.5 * np.linalg.norm(np.cross(edge1, edge2))
            
            # 2. 둘레
            perimeter = (np.linalg.norm(v1 - v0) + 
                        np.linalg.norm(v2 - v1) + 
                        np.linalg.norm(v0 - v2))
            
            # 3. 종횡비 (가장 긴 변 / 가장 짧은 변)
            edge_lengths = [
                np.linalg.norm(v1 - v0),
                np.linalg.norm(v2 - v1),
                np.linalg.norm(v0 - v2)
            ]
            aspect_ratio = max(edge_lengths) / (min(edge_lengths) + 1e-8)
            
            # 4. 각도 (가장 작은 내각)
            angles = []
            for i in range(3):
                v_curr = vertices[face[i]]
                v_prev = vertices[face[(i-1) % 3]]
                v_next = vertices[face[(i+1) % 3]]
                
                vec1 = v_prev - v_curr
                vec2 = v_next - v_curr
                
                cos_angle = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2) + 1e-8)
                cos_angle = np.clip(cos_angle, -1.0, 1.0)
                angle = np.arccos(cos_angle)
                angles.append(angle)
            
            min_angle = min(angles)
            
            # 5. 중심점까지의 거리 (원점에서)
            centroid = (v0 + v1 + v2) / 3.0
            distance_to_origin = np.linalg.norm(centroid)
            
            features.append([area, perimeter, aspect_ratio, min_angle, distance_to_origin])
        
        return np.array(features, dtype=np.float32)
    
    def validate_mesh_data(self, mesh_data: Dict) -> bool:
        """메쉬 데이터 유효성 검사"""
        try:
            vertices = mesh_data['vertices']
            faces = mesh_data['faces']
            
            # 기본 형태 검사
            if vertices.shape[1] != 3 or faces.shape[1] != 3:
                print("❌ 잘못된 데이터 형태")
                return False
            
            # 인덱스 범위 검사
            if faces.max() >= len(vertices) or faces.min() < 0:
                print("❌ 면 인덱스가 정점 범위를 벗어남")
                return False
            
            # NaN/Inf 검사
            if np.any(np.isnan(vertices)) or np.any(np.isinf(vertices)):
                print("❌ 정점에 NaN 또는 Inf 값 존재")
                return False
            
            print("✅ 메쉬 데이터 유효성 검사 통과")
            return True
            
        except Exception as e:
            print(f"❌ 유효성 검사 중 오류: {e}")
            return False

# 사용 예시
if __name__ == "__main__":
    loader = MeshLoader()
    
    # 메쉬 파일 로딩
    try:
        mesh_data = loader.load_mesh_data("example.obj")
        
        if loader.validate_mesh_data(mesh_data):
            print(f"정점 수: {mesh_data['metadata']['vertex_count']}")
            print(f"면 수: {mesh_data['metadata']['face_count']}")
            print(f"특성 차원: {mesh_data['features'].shape}")
        
    except Exception as e:
        print(f"메쉬 로딩 실패: {e}")


In [ ]:
# =============================================================================
# Cell 3: MeshConv Layer 정의
# =============================================================================

class MeshConvLayer(layers.Layer):
    """메쉬 합성곱 레이어"""

    def __init__(self,
                 out_channels: int,
                 kernel_size: int = 5,
                 stride: int = 1,
                 padding: str = 'same',
                 activation: str = 'relu',
                 **kwargs):
        super(MeshConvLayer, self).__init__(**kwargs)

        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.activation = activation

        # 1D 합성곱으로 메쉬의 엣지 특성들을 처리
        self.conv1d = layers.Conv1D(
            filters=out_channels,
            kernel_size=kernel_size,
            strides=stride,
            padding=padding,
            activation=activation,
            name=f'mesh_conv_{out_channels}'
        )

        self.batch_norm = layers.BatchNormalization()

    def call(self, inputs, training=None):
        x = self.conv1d(inputs)
        x = self.batch_norm(x, training=training)
        return x

    def get_config(self):
        config = super().get_config()
        config.update({
            'out_channels': self.out_channels,
            'kernel_size': self.kernel_size,
            'stride': self.stride,
            'padding': self.padding,
            'activation': self.activation
        })
        return config



In [ ]:
# =============================================================================
# Cell 4: MeshPooling Layer 정의
# =============================================================================

class MeshPoolingLayer(layers.Layer):
    """메쉬 풀링 레이어 - 메쉬 해상도를 줄임"""

    def __init__(self,
                 pool_size: int = 2,
                 pool_type: str = 'max',
                 **kwargs):
        super(MeshPoolingLayer, self).__init__(**kwargs)

        self.pool_size = pool_size
        self.pool_type = pool_type

        if pool_type == 'max':
            self.pooling = layers.MaxPooling1D(pool_size=pool_size)
        elif pool_type == 'avg':
            self.pooling = layers.AveragePooling1D(pool_size=pool_size)
        else:
            raise ValueError(f"Unsupported pool_type: {pool_type}")

    def call(self, inputs):
        return self.pooling(inputs)

    def get_config(self):
        config = super().get_config()
        config.update({
            'pool_size': self.pool_size,
            'pool_type': self.pool_type
        })
        return config



In [ ]:
# =============================================================================
# Cell 5: MedMeshCNN 모델 정의
# =============================================================================

class MedMeshCNN(keras.Model):
    """의료 메쉬 분류를 위한 CNN 모델"""

    def __init__(self,
                 num_classes: int,
                 input_features: int = 5,
                 conv_channels: List[int] = [64, 128, 256, 512],
                 fc_features: List[int] = [1024, 512],
                 dropout_rate: float = 0.5,
                 **kwargs):
        super(MedMeshCNN, self).__init__(**kwargs)

        self.num_classes = num_classes
        self.input_features = input_features
        self.conv_channels = conv_channels
        self.fc_features = fc_features
        self.dropout_rate = dropout_rate

        # 메쉬 합성곱 레이어들
        self.mesh_convs = []
        self.mesh_pools = []

        for i, channels in enumerate(conv_channels):
            self.mesh_convs.append(
                MeshConvLayer(
                    out_channels=channels,
                    kernel_size=5,
                    name=f'meshconv_{i+1}'
                )
            )
            self.mesh_pools.append(
                MeshPoolingLayer(
                    pool_size=2,
                    pool_type='max',
                    name=f'meshpool_{i+1}'
                )
            )

        # 전역 풀링
        self.global_pool = layers.GlobalMaxPooling1D()

        # 완전연결 레이어들
        self.fc_layers = []
        self.dropout_layers = []

        for i, features in enumerate(fc_features):
            self.fc_layers.append(
                layers.Dense(
                    features,
                    activation='relu',
                    name=f'fc_{i+1}'
                )
            )
            self.dropout_layers.append(
                layers.Dropout(dropout_rate, name=f'dropout_{i+1}')
            )

        # 분류 레이어
        self.classifier = layers.Dense(
            num_classes,
            activation='softmax' if num_classes > 2 else 'sigmoid',
            name='classifier'
        )

    def call(self, inputs, training=None):
        x = inputs

        # 메쉬 합성곱 및 풀링
        for conv, pool in zip(self.mesh_convs, self.mesh_pools):
            x = conv(x, training=training)
            x = pool(x)

        # 전역 풀링
        x = self.global_pool(x)

        # 완전연결 레이어들
        for fc, dropout in zip(self.fc_layers, self.dropout_layers):
            x = fc(x)
            x = dropout(x, training=training)

        # 분류
        outputs = self.classifier(x)

        return outputs

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_classes': self.num_classes,
            'input_features': self.input_features,
            'conv_channels': self.conv_channels,
            'fc_features': self.fc_features,
            'dropout_rate': self.dropout_rate
        })
        return config

model = MedMeshCNN(num_classes=10)  # 예시: 10개의 클래스로 분류하는 모델
model.summary()

Model: "med_mesh_cnn_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ meshconv_1 (MeshConvLayer)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ meshpool_1 (MeshPoolingLayer)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ meshconv_2 (MeshConvLayer)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ meshpool_2 (MeshPoolingLayer)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ meshconv_3 (MeshConvLayer)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ meshpool_3 (MeshPoolingLayer)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ meshconv_4 (MeshConvLayer)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ meshpool_4 (MeshPoolingLayer)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_4          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc_1 (Dense)                    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc_2 (Dense)                    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classifier (Dense)              │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# =============================================================================
# Cell 6: 모델 빌드 및 컴파일
# =============================================================================

def build_medmeshcnn_model(num_classes: int = 3,
                          input_shape: Tuple = (1800, 5)) -> keras.Model:
    """MedMeshCNN 모델 생성 및 컴파일"""

    # 입력 레이어
    inputs = layers.Input(shape=input_shape, name='mesh_input')

    # MedMeshCNN 모델 인스턴스 생성
    model = MedMeshCNN(
        num_classes=num_classes,
        conv_channels=[64, 128, 256, 512],
        fc_features=[1024, 512],
        dropout_rate=0.5
    )

    # 모델 출력
    outputs = model(inputs)

    # 전체 모델 정의
    full_model = keras.Model(inputs=inputs, outputs=outputs, name='MedMeshCNN')

    # 모델 컴파일
    optimizer = keras.optimizers.Adam(learning_rate=0.001)

    if num_classes > 2:
        loss = 'sparse_categorical_crossentropy'
        metrics = ['accuracy']
    else:
        loss = 'binary_crossentropy'
        metrics = ['accuracy']

    full_model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )

    return full_model

# 모델 생성 예시
model = build_medmeshcnn_model(num_classes=10, input_shape=(1800, 5))
model.summary()


Model: "MedMeshCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mesh_input (InputLayer)         │ (None, 1800, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ med_mesh_cnn_5 (MedMeshCNN)     │ (None, 10)             │     1,921,802 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,921,802 (7.33 MB)

 Trainable params: 1,919,882 (7.32 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [ ]:
# =============================================================================
# Cell 7: 훈련 설정 및 콜백
# =============================================================================

def setup_training_callbacks(model_name: str = 'medmeshcnn',
                           patience: int = 10) -> List:
    """훈련용 콜백 설정"""

    callbacks = [
        # 조기 종료
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=patience,
            restore_best_weights=True,
            verbose=1
        ),

        # 학습률 감소
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),

        # 모델 체크포인트
        keras.callbacks.ModelCheckpoint(
            filepath=f'{model_name}_best.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),

        # CSV 로거
        keras.callbacks.CSVLogger(
            filename=f'{model_name}_training_log.csv',
            separator=',',
            append=False
        )
    ]

    return callbacks


In [ ]:
# =============================================================================
# Cell 8: 데이터 생성 및 전처리 (예시)
# =============================================================================

def generate_sample_data(num_samples: int = 1000,
                        num_classes: int = 3) -> Tuple[np.ndarray, np.ndarray]:
    """샘플 메쉬 데이터 생성 (실제 데이터로 교체 필요)"""

    # 가상의 메쉬 특성 데이터 생성
    X = np.random.randn(num_samples, 1800, 5).astype(np.float32)

    # 랜덤 라벨 생성
    y = np.random.randint(0, num_classes, num_samples)

    print(f"Generated data shape: {X.shape}")
    print(f"Labels shape: {y.shape}")
    print(f"Number of classes: {num_classes}")

    return X, y

# 샘플 데이터 생성
X_sample, y_sample = generate_sample_data(num_samples=1000, num_classes=3)

# 훈련/검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_sample, y_sample,
    test_size=0.2,
    random_state=42,
    stratify=y_sample
)

print(f"Training data: {X_train.shape}, {y_train.shape}")
print(f"Validation data: {X_val.shape}, {y_val.shape}")


Generated data shape: (1000, 1800, 5)
Labels shape: (1000,)
Number of classes: 3
Training data: (800, 1800, 5), (800,)
Validation data: (200, 1800, 5), (200,)


In [ ]:
# =============================================================================
# Cell 9: 모델 훈련
# =============================================================================

def train_medmeshcnn(model: keras.Model,
                    X_train: np.ndarray,
                    y_train: np.ndarray,
                    X_val: np.ndarray,
                    y_val: np.ndarray,
                    epochs: int = 100,
                    batch_size: int = 32) -> keras.callbacks.History:
    """MedMeshCNN 모델 훈련"""

    # 콜백 설정
    callbacks = setup_training_callbacks()

    # 훈련 실행
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1
    )

    return history

# 모델 훈련 실행
print("Starting model training...")
history = train_medmeshcnn(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    epochs=50,
    batch_size=16
)


Starting model training...
Epoch 1/50
48/50 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2925 - loss: 6.0628
Epoch 1: val_accuracy improved from -inf to 0.33000, saving model to medmeshcnn_best.h5


TypeError: Could not locate class 'MeshConvLayer'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'MeshConvLayer', 'config': {'name': 'meshconv_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'out_channels': 64, 'kernel_size': 5, 'stride': 1, 'padding': 'same', 'activation': 'relu'}, 'registered_name': 'MeshConvLayer', 'build_config': {'input_shape': [None, 1800, 5]}}

In [ ]:
# =============================================================================
# Cell 10: 훈련 결과 시각화
# =============================================================================

def plot_training_history(history: keras.callbacks.History):
    """훈련 과정 시각화"""

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # 손실 함수 그래프
    ax1.plot(history.history['loss'], label='Training Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)

    # 정확도 그래프
    ax2.plot(history.history['accuracy'], label='Training Accuracy')
    ax2.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax2.set_title('Model Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

    # 최종 성능 출력
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]

    print(f"Final Training Accuracy: {final_train_acc:.4f}")
    print(f"Final Validation Accuracy: {final_val_acc:.4f}")

# 훈련 히스토리 시각화
plot_training_history(history)


In [ ]:
# =============================================================================
# Cell 11: 모델 평가 및 예측
# =============================================================================

def evaluate_model(model: keras.Model,
                  X_test: np.ndarray,
                  y_test: np.ndarray) -> Dict:
    """모델 평가"""

    # 모델 평가
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

    # 예측 수행
    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)

    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

    return {
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'predictions': y_pred,
        'prediction_probabilities': y_pred_proba
    }

# 검증 데이터로 평가
results = evaluate_model(model, X_val, y_val)


In [ ]:
# =============================================================================
# Cell 12: 모델 저장 및 로딩
# =============================================================================

def save_model(model: keras.Model, model_path: str = 'medmeshcnn_model'):
    """모델 저장"""

    # 전체 모델 저장 (권장)
    model.save(f'{model_path}.h5')

    # SavedModel 형식으로도 저장
    model.save(f'{model_path}_savedmodel')

    print(f"Model saved to {model_path}")

def load_model(model_path: str) -> keras.Model:
    """모델 로딩"""

    # 커스텀 객체와 함께 모델 로딩
    custom_objects = {
        'MeshConvLayer': MeshConvLayer,
        'MeshPoolingLayer': MeshPoolingLayer,
        'MedMeshCNN': MedMeshCNN
    }

    model = keras.models.load_model(model_path, custom_objects=custom_objects)
    print(f"Model loaded from {model_path}")

    return model

# 모델 저장
save_model(model, 'medmeshcnn_tensorflow')


In [ ]:
# =============================================================================
# Cell 13: 추론 및 활용 예시
# =============================================================================

def predict_single_mesh(model: keras.Model,
                       mesh_features: np.ndarray,
                       class_names: List[str] = None) -> Dict:
    """단일 메쉬에 대한 예측"""

    # 배치 차원 추가
    if len(mesh_features.shape) == 2:
        mesh_features = np.expand_dims(mesh_features, axis=0)

    # 예측 수행
    prediction_proba = model.predict(mesh_features)
    predicted_class = np.argmax(prediction_proba, axis=1)[0]
    confidence = np.max(prediction_proba, axis=1)[0]

    result = {
        'predicted_class': predicted_class,
        'confidence': confidence,
        'probabilities': prediction_proba[0]
    }

    if class_names:
        result['predicted_class_name'] = class_names[predicted_class]

    return result

# 예시 추론
sample_mesh = X_val[0]  # 첫 번째 검증 샘플
class_names = ['Class_0', 'Class_1', 'Class_2']  # 실제 클래스명으로 교체

prediction_result = predict_single_mesh(model, sample_mesh, class_names)
print("Prediction Result:")
print(f"Predicted Class: {prediction_result['predicted_class_name']}")
print(f"Confidence: {prediction_result['confidence']:.4f}")
print(f"All Probabilities: {prediction_result['probabilities']}")


In [ ]:
# =============================================================================
# Cell 14: 하이퍼파라미터 튜닝 (선택사항)
# =============================================================================

def hyperparameter_search(X_train, y_train, X_val, y_val):
    """간단한 하이퍼파라미터 검색"""

    param_grid = {
        'conv_channels': [
            [32, 64, 128, 256],
            [64, 128, 256, 512],
            [128, 256, 512, 1024]
        ],
        'dropout_rate': [0.3, 0.5, 0.7],
        'learning_rate': [0.001, 0.0005, 0.0001]
    }

    best_accuracy = 0
    best_params = {}

    for conv_channels in param_grid['conv_channels']:
        for dropout_rate in param_grid['dropout_rate']:
            for lr in param_grid['learning_rate']:

                print(f"Testing: conv_channels={conv_channels}, dropout={dropout_rate}, lr={lr}")

                # 모델 생성
                test_model = build_medmeshcnn_model(num_classes=3)

                # 새로운 옵티마이저로 컴파일
                test_model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=lr),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy']
                )

                # 빠른 훈련 (적은 에포크)
                history = test_model.fit(
                    X_train, y_train,
                    batch_size=16,
                    epochs=10,
                    validation_data=(X_val, y_val),
                    verbose=0
                )

                # 최고 검증 정확도 확인
                best_val_acc = max(history.history['val_accuracy'])

                if best_val_acc > best_accuracy:
                    best_accuracy = best_val_acc
                    best_params = {
                        'conv_channels': conv_channels,
                        'dropout_rate': dropout_rate,
                        'learning_rate': lr
                    }

                print(f"Best validation accuracy: {best_val_acc:.4f}")

    print(f"\nBest parameters: {best_params}")
    print(f"Best accuracy: {best_accuracy:.4f}")

    return best_params

# 하이퍼파라미터 검색 실행 (시간이 오래 걸릴 수 있음)
# best_params = hyperparameter_search(X_train, y_train, X_val, y_val)

print("MedMeshCNN TensorFlow implementation complete!")
print("All cells are ready to run in Jupyter Lab/Notebook environment.")